# Building a Spam Filter with Naive Bayes
## Introduction
The aim in this project is to build a spam filter software, using the multinomial Naive Bayes Algorithm specifically. In essense, the filter would be able to categorise a new message as a spam or not. The standard for succes of this goal would be to developing a filter that is at least 80% accurate.

This process requires a data set to build the algorithm for both training and testing purposes. Consequently, a data set from [UCI](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) would be used. This data set contains 5572 categorised messages and was prepared by Tiago A. Almeida and José María Gómez Hidalgo, and is well sourced for the purpose of our project.

The objectives for the project are as follows:
 * Train the filter software with the training data
 * Test the efficiency of the filter software using the test data
 

## Initial EDA
We begin the project by carrying out an initial exploratory data analytics (EDA) of our data.

In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell  # Configure jupyther to displays...
InteractiveShell.ast_node_interactivity = "all"             # multiple outputs at the same time.

# import data-set
sms_spam = pd.read_csv('Downloads/SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

# General EDA
sms_spam.head()
sms_spam.tail()
sms_spam.describe()
sms_spam.shape
sms_spam['Label'].unique()
sms_spam['Label'].value_counts()
sms_spam['Label'].value_counts(normalize = True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

It appears the 'ham' label is used to categorise non-spam messages , while 'spam' is used for spam messages. Since both 'count' and shape show the same value then there are no instances of np.nans. Finally, among these mesaages, about 87% are non-spam messags while about 13% are spam messages. This sample looks representative, since in practice most messages that people receive are ham.

## Data Split
Next, we proceed to split our data for training and testing purposes and we have decided to apply a 80-20 split, respectively.

In [2]:
# Randomise the dataset
randomised_sample = sms_spam.sample(frac = 1, random_state = 1)

# Split the data set into training and testing data
training_data = randomised_sample.iloc[:(round(0.8 * 5572)-1), :] # 80% for training data
testing_data = randomised_sample.iloc[(round(0.8 * 5572)-1):, :]  # 20% of testing data

# Reset index on both samples
training_data.reset_index(drop = True, inplace = True)
testing_data.reset_index(drop = True, inplace = True)

# Check data split 
print('Percentage distribution of training data:\n',training_data['Label'].value_counts(normalize = True)*100)
print('\n')
print('Percentage distribution of testing_data:\n',testing_data['Label'].value_counts(normalize = True)*100)

Percentage distribution of training data:
 ham     86.53803
spam    13.46197
Name: Label, dtype: float64


Percentage distribution of testing_data:
 ham     86.816143
spam    13.183857
Name: Label, dtype: float64


Above, we split the data into training and testing data, 80% and 20% respectively. Notice that before this split, we randomised the full initial data sample. We did this to facilitate consistency in the proportion of ham and spam (about 87% and 13%) messages post-split, which could have been introduced with the original sorting order of our data sample. 

## Implement Training Data on Software Filter
### Clean Data 
Now we can implement the training data to train the multinomial Naive Bayes algorithm based software filter. The first thing to do is clean the data by removing all punctuation marks and rendering all messages in lowercase. 

In [3]:
# Clean the training data set 
clean_sms = training_data['SMS'].str.replace('\W', ' ').str.lower() 

print('Before cleaning:')
training_data['SMS'].head()

print('After cleaning:')
clean_sms.head()

Before cleaning:


0                         Yep, by the pretty sculpture
1        Yes, princess. Are you going to make me moan?
2                           Welp apparently he retired
3                                              Havent.
4    I forgot 2 ask ü all smth.. There's a card on ...
Name: SMS, dtype: object

After cleaning:


0                         yep  by the pretty sculpture
1        yes  princess  are you going to make me moan 
2                           welp apparently he retired
3                                              havent 
4    i forgot 2 ask ü all smth   there s a card on ...
Name: SMS, dtype: object

As seen above, puctuation marks have been removed and lowercase words have been rendered.

### Transform Data 
In order to implement the Naive Bayes Algorithm on messages data, this data first needs to be transformed into a numeric format, which supports to calculations of the algorithm but still represents the message data. This format is highlighted below: 

![Show picture](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png)

Steps would now be taken to transform our training data.

First, we extract the vocabulary of our training set, which is effectively all the unique words from our training data.

In [4]:
clean_sms = clean_sms.str.strip().str.split(' ').copy()

all_words_all_messages = []                  # Create container that would contains...
                                             # all words from all messages
    
# Define a function to extract all the word from all messages 
def extract_words(lst):
    for each in lst:
        all_words_all_messages.append(each)
    return                                   # return nothing
        
# Run function to extract all the word from all messages
psedo_output_file = clean_sms.apply(extract_words) 

# Create Vocabulary by extracting unique words from 'all_words_all_messages'
vocabulary = set(all_words_all_messages)
vocabulary = list(vocabulary)

# Check results
print('First 10 unique words are:')
vocabulary[:10]
print('Total unique words from out our training data:', len(vocabulary))

First 10 unique words are:


['',
 'jetton',
 'bros',
 'results',
 'jul',
 'dose',
 '0800',
 'lovers',
 'wait',
 'diff']

Total unique words from out our training data: 7783


We have successfully extracted the unique set of words from all messages in our training data in the variable 'vocabulary'. Notice from the output above that an empty string has been characterised as a word. This would be addressed after the transformation is complete.

Now we proceed to transform the trianing data into the format illustrated in the image above, which is a numeric format that supports to calculations of the algorithm but still represents the message data.

In [5]:
# Categorise the frequency of each word in every message
word_counts_per_sms = {unique_word: [0] * 
                       len(clean_sms) for unique_word in vocabulary_list} # Create a container dictionary with...
                                                                          # the correct dimensions
for index, sms in enumerate(clean_sms):
    for word in sms:
        word_counts_per_sms[word][index] += 1
transform_df = pd.DataFrame(word_counts_per_sms)                          # Create transformation...
                                                                          # dataframe 

# Extend the training data by its transformed format
transform_training_data = pd.concat([training_data, transform_df], axis = 1)

# Validate the transformed dataframe
print('Total word from our transformed words frequency table:'
      , transform_df.sum().sum())
print('Unique words from our training data:'
      , len(all_words_all_messages))

# Remove the empty strings that have been characterised as words
transform_training_data.drop(columns  = '', inplace = True) 
transform_training_data.head()

# Remove the empty string as a word from our set of unique words
while '' in vocabulary_list:
    vocabulary_list.remove('')

NameError: name 'vocabulary_list' is not defined

Above, we have successfully created a numeric format which represents messages in our training data and also supports calculations for our multinomial Naive Bayes Algorithm.

It was also important to check the validity of the transformation. We confirmed that the sum of the frequency from our transformation table is equal to the number of all words extracted from all messages ('all_words_all_messages'). 


### Calculate constants
We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:
$$P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)$$
$$P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)$$

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:
$$P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}$$
$$P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}$$

Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:
* P(Spam) and P(Ham)
* NSpam, NHam, NVocabulary

We'll also use Laplace smoothing and set $\alpha = 1$.

In [ ]:
# Calculate the constants of the Naive Bayes Algorithm
alpha = 1 
n_vocabulary = len(vocabulary_list) 

n_of_words_in_spam = transform_training_data[
                                      transform_training_data['Label']
                                      == 
                                      'spam'
                                     ].iloc[:,2:].sum().sum()

n_of_words_in_ham = transform_training_data[
                                     transform_training_data['Label']
                                     == 
                                     'ham'
                                    ].iloc[:,2:].sum().sum()

p_spam = ((training_data['Label'] == 'spam').sum()
          / 
          len(training_data))

p_ham = ((training_data['Label'] == 'ham').sum()
         / 
         len(training_data))

word_count_given_spam_freq_table = transform_training_data[
                                                       transform_training_data['Label'] 
                                                       == 
                                                       'spam'
                                                      ].iloc[:,2:].sum()

word_count_given_ham_freq_table = transform_training_data[
                                                      transform_training_data['Label']
                                                      ==
                                                      'ham'
                                                     ].iloc[:,2:].sum()
print('Total number of unique words in our vocabulary:', n_vocabulary)
print('Total number of words in spam messages:', n_of_words_in_spam)
print('Total number of words in ham messages:', n_of_words_in_ham)
print('Probability of a spam message:', p_spam)
print('Probability of a ham message:', p_ham)
print('\n')
print('Unique word count in all spam messages')
word_count_given_spam_freq_table.head()
print('Unique word count in all ham messages')
word_count_given_ham_freq_table.head()

### Calculate parameters
Above, we have calculated the constants. Now we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.
The parameters are calculated using the formulas:
$$P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}$$
$$P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}$$

    

In [ ]:
# Create container for parameters
p_word_given_spam_dic = {each_word : [0] for each_word in vocabulary_list}
p_word_given_ham_dic = {each_word : [0] for each_word in vocabulary_list}

# Calculate and store parameters
for each_word in vocabulary_list:
    p_word_given_spam_dic[each_word] = ((word_count_given_spam_freq_table[each_word] + alpha) 
                                        / 
                                        (n_of_words_in_spam + (alpha * n_vocabulary)))

    p_word_given_ham_dic[each_word] = ((word_count_given_ham_freq_table[each_word] + alpha) 
                                       /
                                       (n_of_words_in_ham + (alpha * n_vocabulary)))
    
for each in vocabulary_list[:5]:
    print('Given spam,the probability of',each, p_word_given_spam_dic[each])
print('\n')
for each in vocabulary_list[:5]:
    print('Given non_spam,the probability of', each, p_word_given_ham_dic[each])

Above, we calculated all the parameters for out multinomial Naive Bayes Algorithm.  We ended up with two dictionaries with  calculated probabilities for each unique word, from our training data, occuring in either a spam or non spam message. 

With these constants and parameters calculated, we can proceed to define our functions that would categorise messages as either spam or non-spam messages based on these parameters calculated above, using the multinomial Naive Bayes Algorithm.

## Creating the Classifying Function
The spam filter can be understood as a function that:
* Takes in as input a new message (w1, w2, ..., wn).
* Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
* Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
 * If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
 * If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
 * If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.


In [ ]:
import re

# Define function to classify messages as spam or non-spam
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for each in message:
        if each in p_word_given_spam_dic:
            p_spam_given_message *= p_word_given_spam_dic[each]
        if each in p_word_given_ham_dic:
            p_ham_given_message *= p_word_given_ham_dic[each]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'
    
# Test software filter with testing data    
testing_data['predicted'] = testing_data['SMS'].apply(classify_test_set)
testing_data.head()

correct = 0
total = len(testing_data)

for idx, data in testing_data.iterrows():
    if data['Label'] == data['predicted']:
        correct += 1
        
accuracy = correct/total
print('Correct:',correct)
print('Incorrect:', total - correct)
print('From the test run with the testing data, the software filter is',accuracy * 100, 'percent accurate.')

## Build Classifying Function
Above, we defined a function to classify new messages based on the multinomial Naive Bayes Algorithm. We tested the accuracy of the software filter to be 98.74%.

## Conclusion
We tested the accuracy of our software filter based on out testing data and found that the filter works well, with a 98.74% accuracy. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.

Next steps include:
* Analyze the 14 messages that were classified incorrectly and try to figure out why the algorithm classified them incorrectly
* Make the filtering process more complex by making the algorithm sensitive to letter case


id: dismfds
